In [2]:
import numpy as np
from pathlib import Path

In [11]:
file_path = "/home/clewis/wasabi/reaganbullins2/ProjectionProject/rb26/20240111/rb26_20240111_g0/rb26_20240111_g0_imec0/rb26_20240111_g0_t0.imec0.ap.bin"
file_path = Path(file_path)

In [12]:
def readMeta(binFullPath):
    metaName = binFullPath.stem + ".meta"
    metaPath = Path(binFullPath.parent / metaName)
    metaDict = {}
    if metaPath.exists():
        # print("meta file present")
        with metaPath.open() as f:
            mdatList = f.read().splitlines()
            # convert the list entries into key value pairs
            for m in mdatList:
                csList = m.split(sep='=')
                if csList[0][0] == '~':
                    currKey = csList[0][1:len(csList[0])]
                else:
                    currKey = csList[0]
                metaDict.update({currKey: csList[1]})
    else:
        print("no meta file")
    return(metaDict)

In [13]:
meta_data = readMeta(Path(file_path))
meta_data

{'acqApLfSy': '384,384,1',
 'appVersion': '20201103',
 'fileCreateTime': '2024-01-11T15:20:52',
 'fileName': 'D:/reaganbullins/rb26_20240111_g0/rb26_20240111_g0_imec0/rb26_20240111_g0_t0.imec0.ap.bin',
 'fileSHA1': '2C532392359B6870A0E431947A3A35CBAB7302DC',
 'fileSizeBytes': '159684293230',
 'fileTimeSecs': '6912.739966666667',
 'firstSample': '52865617',
 'gateMode': 'Immediate',
 'imAiRangeMax': '0.6',
 'imAiRangeMin': '-0.6',
 'imCalibrated': 'true',
 'imDatApi': '3.31',
 'imDatBs_fw': '2.0.137',
 'imDatBsc_fw': '3.2.176',
 'imDatBsc_hw': '2.1',
 'imDatBsc_pn': 'NP2_QBSC_00',
 'imDatBsc_sn': '386',
 'imDatFx_hw': '1.2',
 'imDatFx_pn': 'NP2_FLEX_0',
 'imDatHs_fw': '5.1',
 'imDatHs_pn': 'NP2_HS_30',
 'imDatHs_sn': '1333',
 'imDatPrb_dock': '1',
 'imDatPrb_pn': 'PRB_1_4_0480_1',
 'imDatPrb_port': '2',
 'imDatPrb_slot': '3',
 'imDatPrb_sn': '19108313841',
 'imDatPrb_type': '0',
 'imLEDEnable': 'false',
 'imMaxInt': '512',
 'imRoFile': 'C:/SpikeGLX/IMRO_tables/1_shank/NPtype0_shank0_g50

In [74]:
# import pickle

In [75]:
# # save out the dictionary
# with open("/home/clewis/repos/realSpike/meta.pkl", "wb") as f:
#     pickle.dump(meta_data, f)

# Get raw data

In [36]:
sRate = float(meta_data['imSampRate'])
print(srate)

30000.0


In [17]:
nChan = int(meta_data['nSavedChans'])
nChan

385

In [18]:
nFileSamp = int(int(meta_data['fileSizeBytes'])/(2*nChan))
print("nChan: %d, nFileSamp: %d" % (nChan, nFileSamp))

nChan: 385, nFileSamp: 207382199


In [19]:
rawData = np.memmap(file_path, dtype='int16', mode='r',
                        shape=(nChan, nFileSamp), offset=0, order='F')

In [20]:
rawData.shape

(385, 207382199)

In [22]:
# Other parameters about what data to read
tStart = 5000        # in seconds
tEnd = 5001
chanList = list(range(0,384))    # list of channels to extract, by index in saved file

In [25]:
firstSamp = int(sRate*tStart)
lastSamp = int(sRate*tEnd)
(firstSamp, lastSamp)


(150000000, 150030000)

In [26]:
selectData = rawData[chanList, firstSamp:lastSamp+1]
selectData.shape

(384, 30001)

In [27]:
type(selectData)

numpy.ndarray

In [29]:
selectData.nbytes / (1024**3)

0.02145838737487793

# Save 1s of analog data 

In [30]:
np.save("/home/clewis/repos/realSpike/analog_data.npy", selectData)

# Convert the analog data to voltage

In [31]:
def OriginalChans(meta):
    if meta['snsSaveChanSubset'] == 'all':
        # output = int32, 0 to nSavedChans - 1
        chans = np.arange(0, int(meta['nSavedChans']))
    else:
        # parse the snsSaveChanSubset string
        # split at commas
        chStrList = meta['snsSaveChanSubset'].split(sep=',')
        chans = np.arange(0, 0)  # creates an empty array of int32
        for sL in chStrList:
            currList = sL.split(sep=':')
            if len(currList) > 1:
                # each set of contiguous channels specified by
                # chan1:chan2 inclusive
                newChans = np.arange(int(currList[0]), int(currList[1])+1)
            else:
                newChans = np.arange(int(currList[0]), int(currList[0])+1)
            chans = np.append(chans, newChans)
    return(chans)

In [32]:
meta_data['snsSaveChanSubset']


'0:383,768'

In [34]:
saved_channels = OriginalChans(meta_data)
saved_channels.shape

(385,)

In [35]:
saved_channels

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

### Channel 768? I think Reagan said this one is for OneBox

# How to convert from analog to voltage

In [47]:
def ChanGainsIM(meta):
    # list of probe types with NP 1.0 imro format
    # np1_imro = [0,1020,1030,1200,1100,1120,1121,1122,1123,1300]
    # number of channels acquired
    acqCountList = meta['acqApLfSy'].split(sep=',')
    APgain = np.zeros(int(acqCountList[0]))     # default type = float64
    LFgain = np.zeros(int(acqCountList[1]))     # empty array for 2.0
    
    if 'imDatPrb_type' in meta:
        probeType = int(meta['imDatPrb_type'])
    else:
        probeType = 0
    
   
    # imro + probe allows setting gain independently for each channel
    imroList = meta['imroTbl'].split(sep=')')
    # One entry for each channel plus header entry,
    # plus a final empty entry following the last ')'        
    for i in range(0, int(acqCountList[0])):
        currList = imroList[i+1].split(sep=' ')
        APgain[i] = float(currList[3])
        LFgain[i] = float(currList[4])
    
    return(APgain, LFgain)

In [48]:
def Int2Volts(meta):
    if 'imMaxInt' in meta:
        maxInt = int(meta['imMaxInt'])
    else:
        maxInt = 512
    fI2V = float(meta['imAiRangeMax'])/maxInt

    return fI2V

In [49]:
Int2Volts(meta_data)

0.001171875

In [64]:
def GainCorrectIM(dataArray, meta):
    # Look up gain with acquired channel ID
    chanList = list(range(0,384))
    chans = OriginalChans(meta)
    APgain, LFgain = ChanGainsIM(meta)
    nAP = len(APgain)
    nNu = nAP * 2

    # Common conversion factor
    fI2V = Int2Volts(meta)

    # make array of floats to return. dataArray contains only the channels
    # in chanList, so output matches that shape
    convArray = np.zeros(dataArray.shape, dtype='float')
    for i in range(0, len(chanList)):
        j = chanList[i]     # index into timepoint
        k = chans[j]        # acquisition index
        if k < nAP:
            conv = fI2V / APgain[k]
        elif k < nNu:
            conv = fI2V / LFgain[k - nAP]
        else:
            conv = 1
        # The dataArray contains only the channels in chanList
        convArray[i, :] = dataArray[i, :]*conv
    return(convArray)

In [65]:
convData = 1e6 * GainCorrectIM(selectData, meta_data)

In [66]:
convData

array([[ 16.40625,  23.4375 ,   0.     , ..., -37.5    , -37.5    ,
        -35.15625],
       [  9.375  ,  -7.03125, -16.40625, ..., -60.9375 , -42.1875 ,
        -32.8125 ],
       [ 39.84375,  23.4375 ,  18.75   , ...,  -4.6875 , -16.40625,
         -7.03125],
       ...,
       [ 35.15625,  39.84375,  37.5    , ...,  42.1875 ,  35.15625,
         39.84375],
       [ 32.8125 ,  30.46875,  35.15625, ...,  30.46875,  35.15625,
         37.5    ],
       [-16.40625, -21.09375, -21.09375, ..., -23.4375 , -25.78125,
        -21.09375]])

In [67]:
convData.shape

(384, 30001)

# Verify conversion

In [68]:
import tifffile

In [69]:
path = "/home/clewis/repos/holo-nbs/rb26_20240111/raw_voltage_chunk.tif"
        # load the data
data = tifffile.memmap(path) # (channels, time)

In [70]:
d = data[:, :30001]

In [71]:
d.shape

(384, 30001)

In [72]:
np.alltrue(convData == d)

True

In [76]:
d

memmap([[ 16.40625,  23.4375 ,   0.     , ..., -37.5    , -37.5    ,
         -35.15625],
        [  9.375  ,  -7.03125, -16.40625, ..., -60.9375 , -42.1875 ,
         -32.8125 ],
        [ 39.84375,  23.4375 ,  18.75   , ...,  -4.6875 , -16.40625,
          -7.03125],
        ...,
        [ 35.15625,  39.84375,  37.5    , ...,  42.1875 ,  35.15625,
          39.84375],
        [ 32.8125 ,  30.46875,  35.15625, ...,  30.46875,  35.15625,
          37.5    ],
        [-16.40625, -21.09375, -21.09375, ..., -23.4375 , -25.78125,
         -21.09375]])

In [77]:
np.save("/home/clewis/repos/realSpike/ground_truth_voltage.npy",d )